# Tutorial HANNAH Framework 
## AutoML School
### Hannover 2024 


### Setup tooling

In [23]:
#!pip install  hannah[vision,models]@git+https://github.com/ekut-es/hannah.git

### Setup library

In [24]:
import hannah
import pathlib
import torch
import os
from contextlib import contextmanager
import sys



# Modify the python module search path to include the current directory
sys.path.append(str(pathlib.Path.cwd()))


@contextmanager
def change_wd(path: pathlib.Path, create: bool = True, clean: bool = False):
    current = pathlib.Path.cwd()
    
    if clean and path.exists():
        import shutil
        shutil.rmtree(path)
        path.mkdir(parents=True)
    
    if create and not path.exists():
        path.mkdir(parents=True)    
    
    try:
        os.chdir(path)
        yield
    finally:
        os.chdir(current)
    
    


### Create Search Space
First we create a simple search space. This search space description is written to a file such that the `hydra` config framework
can refer to it and instantiate the search space. 

The search space consists of a variable number of `conv_relu` blocks with a `linear` layer at the end. 



In [25]:
%%writefile space.py
from hannah.nas.functional_operators.op import ChoiceOp, Tensor, scope
from hannah.nas.functional_operators.operators import AdaptiveAvgPooling, Conv2d, Linear, Relu
from hannah.nas.functional_operators.op import search_space
from hannah.nas.parameters.parameters import CategoricalParameter, IntScalarParameter


@scope
def conv_relu(input, kernel_size, out_channels, stride):
    in_channels = input.shape()[1]
    weight = Tensor(name='weight',
                    shape=(out_channels, in_channels, kernel_size, kernel_size),
                    axis=('O', 'I', 'kH', 'kW'),
                    grad=True)

    conv = Conv2d(stride=stride)(input, weight)
    relu = Relu()(conv)
    return relu


def adaptive_avg_pooling(input):
    return AdaptiveAvgPooling()(input)


def dynamic_depth(*exits, switch):
    return ChoiceOp(*exits, switch=switch)()


def linear(input, out_features):
    input_shape = input.shape()
    in_features = input_shape[1] * input_shape[2] * input_shape[3]
    weight = Tensor(name='weight',
                    shape=(in_features, out_features),
                    axis=('in_features', 'out_features'),
                    grad=True)

    out = Linear()(input, weight)
    return out


@scope
def classifier_head(input, num_classes):
    out = adaptive_avg_pooling(input)
    out = linear(out, num_classes)
    return out


@search_space
def cnn_search_space(name, input, max_blocks=3, max_channels=64, num_classes=10):
    out_channels = IntScalarParameter(16, max_channels, step_size=8, name="out_channels")
    kernel_size = CategoricalParameter([3, 5, 7, 9], name="kernel_size")
    stride = CategoricalParameter([1, 2], name="stride")
    num_blocks = IntScalarParameter(1, max_blocks, name="num_blocks")
    exits = []
    out = input
    for i in range(num_blocks.max + 1):
        out = conv_relu(out,
                        stride=stride.new(),
                        out_channels=out_channels.new(),
                        kernel_size=kernel_size.new())
        exits.append(out)

    out = dynamic_depth(*exits, switch=num_blocks)
    out = classifier_head(out, num_classes=num_classes)
    return out

Overwriting space.py


Now we create a config file for `hydra`:

In [26]:
from omegaconf import OmegaConf
import os

cfg_dict ={"_target_": "space.cnn_search_space",
           "name": "cnn_search_space",
           "num_classes": 10,
           "max_channels": 32,
           "max_blocks": 2}

cfg = OmegaConf.create(cfg_dict)

if not os.path.exists("model"):
    os.mkdir("model")
OmegaConf.save(config=cfg, f="model/cnn_search_space.yaml")


### Define Hardware Description

### Configure NAS

### Run NAS

Now we can run the NAS in the "terminal". Note, that we have to append the current working directory to the PYTHONPATH such 
that the search space located in our current directory can be found. If the model file where to be located in the `hannah/models/`
directory that would not be necessary.

In [28]:
from hannah.tools.train import main
from hydra import compose, initialize_config_module

wd = pathlib.Path.cwd()

with initialize_config_module("hannah.conf", version_base="1.2"):
    cfg = compose(config_name="config", overrides=["model=cnn_search_space", "nas=random_nas", "trainer.max_epochs=1", "dataset=mnist", f"dataset.data_folder={wd / 'data'}", "+trainer.enable_progress_bar=false", "module.num_workers=8"])
    
    with change_wd(wd / "run1", create=True, clean=True):
        main(cfg)


 **                                            **
/**                                           /**
/**       ******   *******  *******   ******  /**
/******  //////** //**///**//**///** //////** /******
/**///**  *******  /**  /** /**  /**  ******* /**///**
/**  /** **////**  /**  /** /**  /** **////** /**  /**
/**  /**//******** ***  /** ***  /**//********/**  /**
//   //  //////// ///   // ///   //  //////// //   //

dataset:
  data_folder: /local/gerum/hannah/tutorials/automlschool24/data
  cls: hannah.datasets.vision.MNISTDataset
  dataset: mnist
  val_percent: 0.1
features:
  _target_: torch.nn.Identity
model:
  _target_: space.cnn_search_space
  name: cnn_search_space
  num_classes: 10
  max_channels: 32
  max_blocks: 1
scheduler:
  _target_: torch.optim.lr_scheduler.OneCycleLR
  max_lr: 0.001
  pct_start: 0.3
  anneal_strategy: cos
  cycle_momentum: true
  base_momentum: 0.85
  max_momentum: 0.95
  div_factor: 25.0
  final_div_factor: 10000.0
  last_epoch: -1
optimizer:
  _tar

KeyboardInterrupt: 

### Evaluate

The results of the neural architecture search are then saved in `run1/history.yml`

In [ ]:
import pandas as pd 
import yaml

history = yaml.unsafe_load((wd / "run1" / "history.yml").read_text())

df = pd.DataFrame((item.result for item in history))
df

,est_act,ff,total_act,total_macs,total_weights,val_error
0,100352.0,0.292995,138874.0,72153568.0,363808.0,0.052310
1,21952.0,0.167889,51098.0,51642720.0,288312.0,0.074898
2,87808.0,0.502111,201626.0,207622656.0,289656.0,0.072520
3,21952.0,0.325369,44730.0,49857856.0,254536.0,0.065048
4,6272.0,0.070293,17386.0,12955456.0,261136.0,0.057575


![](comparison_accuracy_macs.png)

![](comparison_accuracy_memory.png)